In [ ]:
%pip install --upgrade pip
%pip install psycopg2 onnx==1.17.0 onnxruntime==1.19.2 tf2onnx==1.16.1

In [ ]:
from psycopg2 import connect
from os import getenv
from pandas import read_sql_query

try:
    conn = connect(dbname=getenv('DBNAME'), user=getenv('USER'), host=getenv('HOST'), password=getenv('PASSWORD'))
except Exception as e:
    print("I am unable to connect to the database")
    print(e)

sql_query = "SELECT * FROM distance.approaching_vehicle WHERE EVENT_TIMESTAMP < NOW()"
data = read_sql_query(sql_query, con=conn)
conn.close()

In [ ]:
data.head()

In [ ]:
from numpy import random
from sklearn.model_selection import train_test_split

random.default_rng(seed=513421)

train_columns = [
    "other_bottom",
    "other_left",
    "other_right",
    "other_top",
    "other_speed",
    "your_speed"
]
prediction_column = "brake_amount"

x = data[train_columns].values
y = data[prediction_column].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train.shape, x_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=len(train_columns)))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Train the model and get performance
import os
import time

start = time.time()
epochs = 2
history = model.fit(
    x_train,
    y_train,
    epochs=epochs,
    validation_data=(x_test, y_test),
    verbose=True
)
end = time.time()
print(f"Training of model is complete. Took {end-start} seconds")

In [ ]:
import tensorflow as tf
import tf2onnx
import onnx
import pickle
from pathlib import Path

# Normally we use tf2.onnx.convert.from_keras.
# workaround for tf2onnx bug https://github.com/onnx/tensorflow-onnx/issues/2348

# Wrap the model in a `tf.function`
@tf.function(input_signature=[tf.TensorSpec([None, X_train.shape[1]], tf.float32, name='dense_input')])
def model_fn(x):
    return model(x)

# Convert the Keras model to ONNX
model_proto, _ = tf2onnx.convert.from_function(
    model_fn,
    input_signature=[tf.TensorSpec([None, X_train.shape[1]], tf.float32, name='dense_input')]
)

# Save the model as ONNX for easy use of ModelMesh
os.makedirs("models/distance", exist_ok=True)
onnx.save(model_proto, "models/distance/model.onnx")

In [ ]:
type(model_proto)